# PyTorch Tutorial 03: Define Networks
## Overview
In this tutorial, we explain how to define networks. 
在pytorch中定义网络的方式：①sequential模块②复杂网络一般用nn模块

In [1]:
import torch 
import torch.nn as nn


In [2]:
x = torch.rand([8, 100, 10]).detach()#detach()?
x 

tensor([[[0.8981, 0.3694, 0.0370,  ..., 0.9766, 0.0271, 0.8501],
         [0.5021, 0.7461, 0.1581,  ..., 0.7167, 0.1369, 0.2732],
         [0.4663, 0.6417, 0.3524,  ..., 0.9466, 0.0888, 0.0839],
         ...,
         [0.9165, 0.6616, 0.6528,  ..., 0.1103, 0.6601, 0.4323],
         [0.1486, 0.8005, 0.7195,  ..., 0.7407, 0.0810, 0.5490],
         [0.8683, 0.5279, 0.6342,  ..., 0.4721, 0.1308, 0.0269]],

        [[0.8770, 0.4527, 0.1853,  ..., 0.9056, 0.5798, 0.9272],
         [0.5516, 0.6729, 0.4450,  ..., 0.9623, 0.0930, 0.3770],
         [0.5665, 0.0622, 0.8109,  ..., 0.1841, 0.5201, 0.7251],
         ...,
         [0.0133, 0.1244, 0.1372,  ..., 0.1211, 0.2357, 0.2388],
         [0.5162, 0.1379, 0.0815,  ..., 0.9151, 0.2089, 0.7587],
         [0.9097, 0.1936, 0.3139,  ..., 0.8380, 0.8295, 0.0303]],

        [[0.2070, 0.0439, 0.5768,  ..., 0.6182, 0.7775, 0.6866],
         [0.1776, 0.8487, 0.7369,  ..., 0.7675, 0.0982, 0.5767],
         [0.1524, 0.8180, 0.0434,  ..., 0.6283, 0.2910, 0.

In [3]:
y = torch.rand(8)
y = (y>0.5).int()#大于0.5的变1，小于0.5的变0
y

tensor([0, 1, 1, 1, 1, 0, 0, 1], dtype=torch.int32)

In [7]:
#建议每一行运行一下，了解每个量的输入和输出是什么
class MLP(nn.Module):#继承nn.Module;MLP:多层感知网络（全连接层）
    def __init__(self):
        super(MLP, self).__init__()
        self.first_layer = nn.Linear(1000,50)#输入1000维，输出50维
        self.second_layer = nn.Linear(50, 1)
        #为什么把网络写在_init_()中？init中注册了之后，才知道哪些参数进行优化
        #把多个线性层反复直接堆起来？（如多个first_layer堆起来）权重没有变化
        #最终的维度与y有关，二分类问题，最后转换成一维概率即可;如果是多分类问题，比如3分类，那么最终的layer的输出实际上应该是3
    def forward(self, x):#x：每个batch的输入
        x = torch.flatten(x, start_dim=1, end_dim=2)#把x的后面两个维度拉直，变成了1000维的向量
        x = nn.functional.relu(self.first_layer(x))#并没有把relu做init，因为至少标准的relu中，没有需要求解的参数或权重
        #假设relu中还有一些参数需要我们求解，则需要先进行初始化，否则无初始化且不会进行恰当优化
        x = self.second_layer(x)
        return x

In [5]:
mlp = MLP()#实例化这个网络能不能也很简单？
output = mlp(x)

In [6]:
output#output基本随机：没有强调参数，也没有刻意初始化——默认的初始化；因为是线性层，所以量纲理论上是正负无穷之间，这里因为初始化原因，都接近0
#如果要转换为概率，可以做一个Logit或sofrmax的操作，这里并没有做，会最终在loss函数中去做

tensor([[-0.1922],
        [-0.1654],
        [-0.2327],
        [-0.2706],
        [-0.2534],
        [-0.2508],
        [-0.2221],
        [-0.2470]], grad_fn=<AddmmBackward0>)

In [8]:
class Embedding(nn.Module):#一个把对应的如012345678以及到输入有多少个这样的词汇或者多少个Entity，应用成多少维
    def __init__(self):
        super(Embedding, self).__init__()
        self.embedding = nn.Embedding(4, 100)#输入词汇串中最大的数值（字母？）：只能由数字0、1、2、3构成，不能是4、5、6...，每个词汇对应为100维向量(embbeding的dim)
    def forward(self, x):
        return self.embedding(x)

In [10]:
embedding = Embedding()

embedding_input = torch.tensor([[0,1, 0],[2,3, 3]])
embedding_output = embedding(embedding_input)

In [13]:
embedding_output.shape

torch.Size([2, 3, 100])

In [14]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(10, 
                           15, 
                           num_layers=2, 
                           bidirectional=True, 
                           dropout=0.1)#10：Embbeding的dim；15：隐藏层的维度（hidden dim）；
        #layer：一共把这LSTM的多少层（多层叠加：下面LSTM的隐藏层的输出等于上面一层LSTM的输入，对传统词向量该层数可能是2，最多是3，如果是BERT作为LSTM的输入的话，多层也无实际帮助）
        #bidirectional：在进行文本操作时，需要对其方向做一个指定，因为有时候语义不仅取决于我之前说了什么，也取决于之后说的什么，当该值=true时，我既要将该向量按照前向后输入一遍，接着还要从后向前输一遍
        #dropout：最后一层的dropout
    def forward(self, x):
        output, (hidden, cell) = self.lstm(x)
        return output, hidden, cell#三个输出：output是它每一个位置的hidden，最终一层它每一个timestep的输出，而hidden只是最终状态的输出，cell是hidden中的一些状态
        #我们最终有4个hidden：num_layers=2,bidirectional=True
       

In [23]:
permute_x = x.permute([1,0,2])
 #MLP中，我们假设第一维是观测数，而在LSTM中实际上给它观测的是第二位，因此要对MLP做一个permute操作才能和LSTM保持一致：把它的第二维移成第一维，第三维保持不动
lstm = LSTM()
output_lstm1, output_lstm2, output_lstm3 = lstm(permute_x)

In [28]:
output_lstm1.shape#output_lstm1第一维是序列长度，第二维才是观测数batch size——permute,30=15*2，往前和往后拿15
#output_lstm2第二维是batch size没有变，因为是最终的输出，实际返回不止最后一层，还包括之前层，第一个维度是4（4个hidden）, 15=hidden dim

torch.Size([100, 8, 30])

In [13]:
class Conv(nn.Module):#convolution
    def __init__(self):
        super(Conv, self).__init__()
        self.conv1d = nn.Conv1d(100, 50, 2)#1d层数，100：in_channel；50：out_channel；2：kernel；卷积是二维的，所以kernel_size=2
    def forward(self, x):
        return self.conv1d(x)

In [14]:
conv = Conv()
output = conv(x)

In [15]:
output.shape#channel的数量从100变成50

torch.Size([8, 50, 9])

最复杂的问题：维度——把其中的一些层拿出来进行测试